# 식품

In [1]:
import numpy as np
import pandas as pd
import datetime as datetime
# import dataload
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
tqdm.pandas()

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


## 0. 데이터 로드

In [11]:
cd /content/drive/MyDrive/2021/날씨 공모전/code

/content/drive/MyDrive/2021/날씨 공모전/code


### 0-1. 구매이력

In [13]:
food_cluster = pd.read_csv('../data/food_tsclustering_norm.csv', encoding='cp949')

In [14]:
food_cluster = food_cluster[['sm_cat', 'cluster']]
food_cluster.head()

,sm_cat,cluster
0,가공란,3
1,가자미,3
2,갈비/찜/바비큐용 돈육,3
3,갈비용 우육,3
4,갈치,1


In [15]:
buy_full = pd.read_csv('../data/data.csv', encoding='cp949')

In [16]:
buy_full['date'] = buy_full['date'].apply(lambda x : pd.to_datetime(x, format='%Y%m%d'))
buy_full['year'] = buy_full['date'].apply(lambda x : x.year)

In [17]:
buy_full.head()

,date,sex,age,big_cat,sm_cat,qty,cnt,year
0,2018-01-01,F,20,식품,가공란,37,0.480964,2018
1,2018-01-01,F,30,식품,가공란,16,0.480964,2018
2,2018-01-01,F,40,식품,가공란,9,0.480964,2018
3,2018-01-01,F,50,식품,가공란,3,0.480964,2018
4,2018-01-01,M,20,식품,가공란,13,0.480964,2018


In [19]:
buy1819 = buy_full[(buy_full.year == 2018)|(buy_full.year == 2019)]

In [20]:
food1819 = buy1819[buy1819.big_cat=='식품']

In [21]:
food1819.head()

,date,sex,age,big_cat,sm_cat,qty,cnt,year
0,2018-01-01,F,20,식품,가공란,37,0.480964,2018
1,2018-01-01,F,30,식품,가공란,16,0.480964,2018
2,2018-01-01,F,40,식품,가공란,9,0.480964,2018
3,2018-01-01,F,50,식품,가공란,3,0.480964,2018
4,2018-01-01,M,20,식품,가공란,13,0.480964,2018


In [22]:
food = food1819.merge(food_cluster, on='sm_cat')
food.head()

,date,sex,age,big_cat,sm_cat,qty,cnt,year,cluster
0,2018-01-01,F,20,식품,가공란,37,0.480964,2018,3
1,2018-01-01,F,30,식품,가공란,16,0.480964,2018,3
2,2018-01-01,F,40,식품,가공란,9,0.480964,2018,3
3,2018-01-01,F,50,식품,가공란,3,0.480964,2018,3
4,2018-01-01,M,20,식품,가공란,13,0.480964,2018,3


In [23]:
food0 = food[food.cluster==0]
food1 = food[food.cluster==1]
food2 = food[food.cluster==2]
food3 = food[food.cluster==3]
food4 = food[food.cluster==4]

In [34]:
food0.to_csv('../data/food0.csv', encoding='cp949', index=None)
food1.to_csv('../data/food1.csv', encoding='cp949', index=None)
food2.to_csv('../data/food2.csv', encoding='cp949', index=None)
food3.to_csv('../data/food3.csv', encoding='cp949', index=None)
food4.to_csv('../data/food4.csv', encoding='cp949', index=None)

### 0-2. 날씨

In [27]:
weather = pd.read_csv('../data/weather_final.csv', encoding='cp949')
weather['날짜'] = weather['날짜'].apply(lambda x : pd.to_datetime(x, format='%Y-%m-%d'))

weather.head()

,날짜,지점번호,평균기온,최고기온,최저기온,일별강수량,1시간최대강수량,평균풍속,지역,PM10,PM25,1시간최대습도,일조시간합,최고현지기압,연,월,일,분기,요일,공휴일명,공휴일여부,주말여부,계절,체감온도,열지수,폭염여부,강수여부
0,2018-01-01,105,1.3,5.7,-2.1,0.0,0.0,3.7,강릉,20.066667,13.400000,25.4,57.9,1023.0,2018,1,1,1,0,1월1일,1,0,3,7.502056,-39.701524,0,0
1,2018-01-01,112,-0.3,2.7,-2.7,0.0,0.0,1.6,인천,37.518681,18.641758,67.2,53.8,1020.3,2018,1,1,1,0,1월1일,1,0,3,9.992379,-42.924922,0,0
2,2018-01-01,119,-1.7,4.7,-6.9,0.0,0.0,1.0,수원,42.782895,21.375000,84.7,52.7,1025.1,2018,1,1,1,0,1월1일,1,0,3,10.136402,-45.791893,0,0
3,2018-01-01,136,-1.0,4.7,-6.5,0.0,0.0,2.2,안동,39.750000,28.583333,56.4,58.3,1010.6,2018,1,1,1,0,1월1일,1,0,3,8.356692,-44.375704,0,0
4,2018-01-01,152,2.1,6.2,-0.4,0.0,0.0,3.3,울산,38.548193,15.433735,42.8,58.3,1017.1,2018,1,1,1,0,1월1일,1,0,3,8.861429,-38.064334,0,0


## 1. 클러스터별 상관관계 분석

In [28]:
def cluster_buy_df(c) : 
    clusterdf = food[food["cluster"] == c]
    df = buy_full[buy_full.sm_cat.isin(list(clusterdf["sm_cat"]))] 
    df = df.groupby('date').sum('qty').reset_index()[["date","qty"]]
    return df

def weather_correlation(buy_df, loc) : 
    df = weather[weather.지역 == loc].reset_index(drop=True)
    df = df.merge(buy_df, left_on="날짜", right_on="date")
    df = df.drop(['연', '월', '일', '분기', '요일', '공휴일명', '공휴일여부', '주말여부', '계절', '폭염여부', '강수여부'], axis=1)
    co = dict(df.corr()["qty"])
    return co

def display_weather_correlation(buy_df) : 
    weathercorrelation = dict()
    for loc in weather.지역.unique() : 
        weathercorrelation[loc] = weather_correlation(buy_df, loc)
    
    tmp = pd.DataFrame(weathercorrelation)
    tmp = tmp.drop(['지점번호', 'qty'], axis=0)
    return tmp

### 1-1. 클러스터 0

In [29]:
display_weather_correlation(cluster_buy_df(0))

,강릉,인천,수원,안동,울산,제주,천안,청주,대구,창원,여수,부산,서울,광주,전주,대전
평균기온,-0.366790,-0.387113,-0.357695,-0.331241,-0.375820,-0.401170,-0.337503,-0.353202,-0.349535,-0.371177,-0.403223,-0.406582,-0.359183,-0.348445,-0.356665,-0.346199
최고기온,-0.357407,-0.381265,-0.347958,-0.318743,-0.367022,-0.381244,-0.332274,-0.342468,-0.333955,-0.347982,-0.398126,-0.410381,-0.353688,-0.327600,-0.340424,-0.338031
최저기온,-0.382900,-0.385582,-0.354685,-0.349742,-0.378613,-0.411871,-0.340319,-0.360454,-0.361441,-0.380713,-0.405220,-0.402548,-0.359679,-0.359586,-0.363013,-0.355933
일별강수량,-0.056724,-0.020726,-0.007368,-0.012221,-0.051704,-0.061521,-0.029117,-0.024859,-0.040198,-0.057678,-0.042804,-0.050470,-0.002748,-0.038298,-0.027521,-0.025397
1시간최대강수량,-0.059586,-0.039982,-0.020430,-0.020567,-0.050339,-0.068497,-0.023528,-0.026925,-0.038962,-0.058978,-0.041090,-0.045168,-0.012385,-0.030163,-0.036527,-0.032255
평균풍속,0.218766,0.151222,0.099253,0.229133,0.205466,0.135792,0.162734,0.194595,0.196434,0.141170,0.108139,0.197998,0.045810,0.067588,0.165317,0.156872
PM10,0.142331,0.159282,0.193320,0.340023,0.217254,0.141830,0.156644,0.181334,0.209628,0.196448,0.153172,0.276609,0.220676,0.153354,0.196852,0.265648
PM25,0.221017,0.169107,0.147825,0.457785,0.140121,0.131566,0.110469,0.182952,0.169913,0.185648,0.162872,0.214681,0.191257,0.158713,0.147320,0.176987
1시간최대습도,-0.276563,-0.040186,-0.102568,-0.247885,-0.219700,-0.045467,-0.101600,-0.062354,-0.237487,-0.186265,-0.102387,-0.203220,-0.087450,-0.147121,-0.224739,-0.166799
일조시간합,0.054387,-0.027230,0.005736,0.106084,0.041338,-0.049616,-0.010101,0.016443,0.052772,0.029165,0.006323,0.033811,-0.004696,-0.010298,0.012239,-0.008293


### 1-2. 클러스터 1

In [30]:
display_weather_correlation(cluster_buy_df(1))

,강릉,인천,수원,안동,울산,제주,천안,청주,대구,창원,여수,부산,서울,광주,전주,대전
평균기온,-0.042136,-0.054539,-0.050439,-0.047247,-0.043227,-0.058014,-0.053206,-0.048551,-0.041757,-0.047651,-0.057410,-0.053020,-0.050048,-0.062528,-0.056539,-0.049427
최고기온,-0.043236,-0.059052,-0.055893,-0.051019,-0.041677,-0.060585,-0.060336,-0.056150,-0.044786,-0.042604,-0.059619,-0.055221,-0.052080,-0.069548,-0.057547,-0.053845
최저기온,-0.040651,-0.048148,-0.045261,-0.050504,-0.047229,-0.052194,-0.055425,-0.044996,-0.045508,-0.051370,-0.058411,-0.053254,-0.049012,-0.056304,-0.055921,-0.047868
일별강수량,0.034835,0.077052,0.074528,0.061516,-0.028270,0.032131,0.007893,0.055725,-0.009914,-0.004833,-0.002717,-0.036325,0.044377,-0.018594,-0.007317,0.029709
1시간최대강수량,0.050730,0.086374,0.103005,0.107112,-0.013103,0.098407,0.053735,0.112262,0.006597,0.013507,0.002854,-0.026050,0.049328,-0.012855,0.008822,0.078894
평균풍속,0.028167,0.064709,0.052735,0.031840,-0.017675,0.007413,0.017614,0.014079,0.034947,-0.051514,0.058697,0.016635,0.117040,0.015106,0.022727,0.022478
PM10,0.014346,0.043725,0.039293,-0.001815,0.017689,-0.046636,0.034342,0.008187,0.026961,0.011124,-0.021354,0.020314,0.046512,0.005594,-0.004999,0.019838
PM25,0.000084,0.007994,0.014093,-0.028517,-0.009478,-0.058106,0.009392,-0.027430,0.011966,-0.010972,-0.031017,0.016954,0.012510,-0.023696,-0.025671,-0.006474
1시간최대습도,-0.072277,0.005228,-0.025675,-0.055539,-0.077137,-0.013487,0.005111,-0.013622,-0.043685,-0.081465,-0.038491,-0.065400,-0.027156,-0.024710,-0.076547,-0.037932
일조시간합,-0.018611,-0.035195,-0.019590,-0.026380,0.006767,-0.056157,-0.048213,-0.054862,-0.014909,0.011558,-0.019286,0.005720,-0.018024,-0.030540,-0.045326,-0.059616


### 1-3. 클러스터 2

In [31]:
display_weather_correlation(cluster_buy_df(2))

,강릉,인천,수원,안동,울산,제주,천안,청주,대구,창원,여수,부산,서울,광주,전주,대전
평균기온,0.248649,0.241331,0.248967,0.251645,0.242436,0.216172,0.244967,0.251981,0.254884,0.238196,0.221024,0.224106,0.249326,0.228452,0.235970,0.249762
최고기온,0.243973,0.235073,0.241420,0.238338,0.239833,0.212543,0.235155,0.243510,0.246886,0.240814,0.219477,0.213767,0.243428,0.216842,0.230761,0.241211
최저기온,0.241364,0.244512,0.243753,0.236778,0.238359,0.222473,0.227184,0.247066,0.246595,0.230192,0.217668,0.224147,0.244277,0.232958,0.231030,0.242576
일별강수량,0.015389,0.075312,0.058030,0.042456,0.001529,0.066515,-0.002610,0.034890,0.007169,0.029260,0.039082,0.002833,0.065027,-0.002765,0.004347,0.011456
1시간최대강수량,0.041524,0.093213,0.100913,0.088920,0.036949,0.142522,0.050264,0.085846,0.035221,0.057936,0.054960,0.007669,0.090375,0.006649,0.037015,0.057531
평균풍속,-0.069880,-0.025070,0.070020,0.026999,-0.132692,-0.063280,0.026489,0.051653,0.008995,-0.102067,-0.039896,0.029442,0.064852,0.017761,-0.008589,0.051599
PM10,0.005368,-0.046177,-0.047263,-0.083092,0.006049,-0.065635,-0.093933,-0.112902,-0.058625,-0.038694,-0.056378,-0.025484,-0.042382,-0.079511,-0.122071,-0.114062
PM25,-0.073126,-0.082136,-0.087307,-0.175067,-0.003965,-0.078801,-0.107887,-0.155692,-0.088812,-0.058677,-0.081549,-0.020365,-0.069727,-0.119047,-0.155098,-0.137917
1시간최대습도,0.072350,0.137027,0.011371,-0.006555,0.030606,0.140189,0.080201,-0.000563,0.023994,0.069506,0.155063,0.117940,0.062723,0.092857,0.050694,0.011720
일조시간합,0.026950,0.007193,0.002477,0.016861,0.017892,0.027612,0.007877,-0.006142,-0.019299,0.009472,-0.023836,0.012997,0.017044,-0.022929,-0.012362,-0.005013


### 1-4. 클러스터 3

In [32]:
display_weather_correlation(cluster_buy_df(3))

,강릉,인천,수원,안동,울산,제주,천안,청주,대구,창원,여수,부산,서울,광주,전주,대전
평균기온,0.078705,0.059930,0.059483,0.056319,0.068986,0.047974,0.052362,0.068899,0.074019,0.060952,0.058124,0.063056,0.063812,0.043239,0.050281,0.062490
최고기온,0.092485,0.061478,0.060383,0.053979,0.078585,0.050060,0.055656,0.067190,0.078573,0.070803,0.064626,0.063550,0.067244,0.039265,0.051463,0.062877
최저기온,0.067342,0.058170,0.048065,0.041287,0.054345,0.051106,0.036951,0.059877,0.058725,0.051212,0.054586,0.062653,0.052765,0.043308,0.046365,0.052710
일별강수량,0.037557,0.011727,0.005011,0.029200,0.007120,0.023384,-0.000356,0.007568,-0.010009,0.006326,-0.006714,-0.020252,0.014661,-0.024163,-0.006873,0.002528
1시간최대강수량,0.045049,0.029890,0.021557,0.051752,0.015018,0.057319,0.037816,0.028244,-0.003293,0.015954,0.001972,-0.014999,0.022374,-0.021971,0.016199,0.033184
평균풍속,-0.021848,-0.001272,-0.011640,-0.035800,-0.093948,-0.036230,-0.080209,-0.133943,-0.042032,-0.090731,-0.010428,-0.008841,0.101939,-0.013910,-0.013518,-0.066738
PM10,0.017563,0.078474,0.064046,-0.042108,-0.029654,-0.005245,0.076470,0.008064,-0.014039,-0.037703,-0.023494,-0.044851,0.061714,0.040219,-0.019680,-0.020962
PM25,-0.004943,0.051472,0.056812,-0.161643,-0.037584,-0.010153,0.085151,-0.004013,-0.004054,-0.026498,-0.021623,-0.016946,0.053500,0.008723,-0.003636,0.012333
1시간최대습도,0.018201,0.053811,0.035561,-0.016199,0.021442,-0.037330,0.103352,-0.008364,0.043308,-0.025244,0.007944,0.026545,0.000641,0.001355,-0.026720,-0.014130
일조시간합,0.025045,0.005096,-0.009360,-0.014993,0.032423,0.026591,-0.031057,-0.041767,-0.003553,0.055896,0.009493,0.031741,0.012747,-0.012734,-0.026069,-0.023661


### 1-5. 클러스터 4

In [33]:
display_weather_correlation(cluster_buy_df(4))

,강릉,인천,수원,안동,울산,제주,천안,청주,대구,창원,여수,부산,서울,광주,전주,대전
평균기온,-0.656999,-0.682632,-0.702959,-0.713805,-0.668740,-0.650451,-0.713557,-0.705520,-0.697106,-0.684977,-0.667980,-0.656840,-0.701272,-0.703462,-0.697128,-0.707995
최고기온,-0.631436,-0.684825,-0.702760,-0.697976,-0.643659,-0.651726,-0.703855,-0.705994,-0.674598,-0.672382,-0.658503,-0.633866,-0.696338,-0.698042,-0.698071,-0.700668
최저기온,-0.648798,-0.682154,-0.691937,-0.691219,-0.671010,-0.639718,-0.687375,-0.690816,-0.689272,-0.676870,-0.659241,-0.656912,-0.694918,-0.681747,-0.680138,-0.689736
일별강수량,-0.072958,-0.123713,-0.104631,-0.106200,-0.090351,-0.061809,-0.101640,-0.111116,-0.117390,-0.108994,-0.106041,-0.116158,-0.142147,-0.120972,-0.125533,-0.122556
1시간최대강수량,-0.085053,-0.127917,-0.119063,-0.109927,-0.117706,-0.075464,-0.108501,-0.106208,-0.138760,-0.114059,-0.126607,-0.127247,-0.160171,-0.123884,-0.137472,-0.120867
평균풍속,0.321524,0.151006,-0.129870,-0.037271,0.145197,0.165485,-0.189010,-0.277039,-0.020211,0.003083,0.147956,-0.098014,0.142234,-0.116320,0.036835,-0.219205
PM10,0.058332,0.294438,0.273963,0.182649,0.063718,0.074921,0.350876,0.328739,0.281820,0.145880,0.121376,0.115729,0.279974,0.239467,0.281209,0.323721
PM25,0.142997,0.226017,0.260273,0.235571,-0.001346,0.055185,0.304490,0.314124,0.292507,0.112145,0.103294,0.114913,0.223272,0.230534,0.310086,0.320099
1시간최대습도,-0.390579,-0.307641,-0.138609,-0.223291,-0.412152,-0.352157,-0.115561,-0.051736,-0.247240,-0.472151,-0.484273,-0.463085,-0.233690,-0.309942,-0.338234,-0.162354
일조시간합,0.008038,-0.090459,-0.071783,-0.080705,-0.014512,-0.241750,-0.131032,-0.112770,0.012449,0.048756,-0.039088,0.019975,-0.099569,-0.052347,-0.082924,-0.106538
